In [1]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop\GraphicalPR


In [2]:
import numpy as np
from core.uncertain_array import UncertainArray as UA
from graph.prior import SparsePrior, GaussianPrior
from graph.propagator.unitary_propagator import UnitaryPropagator
from graph.measurement import MaskedGaussianMeasurement, GaussianMeasurement
from core.linalg_utils import random_unitary_matrix
from core.linalg_utils import random_binary_mask

## SparsePrior & MaskedGaussianMeasurement

In [3]:
# 1. パラメータと真のスパース信号
n = 512
sparsity = 0.1
noise_var = 1e-4
mask_ratio = 0.2
np.random.seed(15)

true_x = np.zeros(n, dtype=np.complex128)
nonzero_indices = np.random.choice(n, int(n * sparsity), replace=False)
true_x[nonzero_indices] = (np.random.randn(len(nonzero_indices)) + 1j * np.random.randn(len(nonzero_indices)))*np.sqrt(1/2)

# 2. ユニタリ行列と観測
U = random_unitary_matrix(n, seed=3)
true_y = U @ true_x
noise = (np.random.randn(n) + 1j * np.random.randn(n)) * np.sqrt(noise_var/2)
observed = true_y + noise
mask = random_binary_mask(n, subsampling_rate= mask_ratio, seed=None)

In [4]:
# 3. グラフ構築
X = SparsePrior(rho=sparsity, shape=(n,), damping=0.03)
Y = UnitaryPropagator(U) @ X
Z = MaskedGaussianMeasurement(Y, observed, var=noise_var, mask = mask)

In [5]:
# 4. 推論ステップ（最小：1往復）
for t in range(20):
    X.parent.forward()
    X.forward()
    Y.parent.forward()
    Y.forward()
    Z.forward()
    Z.backward()
    Y.backward()
    Y.parent.backward()
    X.backward()
    X.parent.backward()
    if t % 5 == 0:
        x_belief = X.compute_belief()
        print("t = ", t, "Reconstruction MSE:", np.mean(np.abs(x_belief.data - true_x) ** 2))

t =  0 Reconstruction MSE: 0.4124600700840233
t =  5 Reconstruction MSE: 0.04990736956084199
t =  10 Reconstruction MSE: 0.01804563784554417
t =  15 Reconstruction MSE: 0.0014384438105604553
